# geDIG Attention Update (Modern LLMs)

Goal: sample attention maps from newer LLMs (Llama 3, Phi-3) and check whether geDIG F keeps the same sign/delta vs. random.

Notes:
- Llama 3 requires an HF token and license acceptance.
- Phi-3 is open and should run without a token.


In [ ]:
from google.colab import drive

drive.mount('/content/drive')


In [ ]:
!git clone https://github.com/miyauchikazuyoshi/InsightSpike-AI.git
%cd InsightSpike-AI
!pip -q install -U pip
!pip -q install transformers datasets accelerate sentencepiece networkx scipy huggingface_hub


In [ ]:
from huggingface_hub import login

# login()  # Uncomment and paste your HF token if you want to use Llama 3


In [ ]:
from huggingface_hub import HfApi
from IPython.display import display, HTML

LLAMA3_ID = "meta-llama/Meta-Llama-3-8B"
PHI3_ID = "microsoft/Phi-3-mini-4k-instruct"

def show_status(ok: bool, title: str, detail: str = ""):
    color = "#1b8a5a" if ok else "#b02a37"
    msg = f"<b>{title}</b>" + (f"<br/>{detail}" if detail else "")
    display(HTML(f"<div style='border:1px solid {color};padding:10px;border-radius:6px;color:{color};margin:8px 0;'>{msg}</div>"))

api = HfApi()
llama3_ok = False
try:
    api.model_info(LLAMA3_ID)
    show_status(True, "Llama 3 access OK", f"Model: {LLAMA3_ID}")
    llama3_ok = True
except Exception as exc:
    show_status(False, "Llama 3 access NOT available", f"{type(exc).__name__}: {exc}")
    print("If you need Llama 3, accept the license on HF and run login().")


In [ ]:
DTYPE = "bfloat16"  # use "float16" for T4, "float32" for CPU
ATTN_IMPL = "eager"  # to ensure attentions are returned
TEXT_COUNT = 16
LAYER_CAP = 4
ATTN_MAX_LEN = 256


In [ ]:
import json
import numpy as np
from pathlib import Path

outputs = {}


def run_model(model_id: str, tag: str):
    out_path = f"results/transformer_gedig/score_llm_{tag}.json"
    cmd = (
        f"python experiments/transformer_gedig/extract_and_score.py "
        f"--model "{model_id}" "
        f"--text-count {TEXT_COUNT} "
        f"--layer-cap {LAYER_CAP} "
        f"--attn-max-len {ATTN_MAX_LEN} "
        f"--percentile 0.90 "
        f"--out "{out_path}" "
        f"--device auto --dtype {DTYPE} --device-map auto "
        f"--trust-remote-code --attn-implementation {ATTN_IMPL}"
    )
    print(cmd)
    !{cmd}
    outputs[tag] = out_path


# Always run Phi-3
run_model(PHI3_ID, "phi3")

# Run Llama 3 only if access is available
if llama3_ok:
    run_model(LLAMA3_ID, "llama3")
else:
    print("Skipping Llama 3 run (no access).")


In [ ]:
def summarize(path):
    data = json.loads(Path(path).read_text(encoding="utf-8"))
    rows = [r for r in data if not r.get("subgraph")]
    if not rows:
        return None
    f_real = np.array([r["F"] for r in rows], dtype=float)
    f_rand = np.array([r.get("baseline_F_random") for r in rows], dtype=float)
    delta = f_real - f_rand
    return {
        "rows": len(rows),
        "F_real_mean": float(f_real.mean()),
        "F_rand_mean": float(f_rand.mean()),
        "delta_mean": float(delta.mean()),
        "delta_std": float(delta.std()),
        "delta_positive_ratio": float((delta > 0).mean()),
    }

summary = {tag: summarize(path) for tag, path in outputs.items()}
print(json.dumps(summary, indent=2))


In [ ]:
import shutil

save_dir = Path("/content/drive/MyDrive/insightspike/gedig_attention")
save_dir.mkdir(parents=True, exist_ok=True)

for tag, path in outputs.items():
    src = Path(path)
    if src.exists():
        shutil.copy2(src, save_dir / src.name)
        print("Saved:", save_dir / src.name)
